In [61]:
import cv2
import sys
import numpy as np
from cv2 import imshow as show

In [6]:
cellStates = ['empty','present','absent']

In [7]:
# to rotate an image 
def rotate(img, angle):
    rows = img.shape[0]
    cols = img.shape[1]

    img_center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(img_center, angle, 1)

    rotated_image = cv2.warpAffine(img, M, (cols, rows),
                    borderMode=cv2.BORDER_CONSTANT,
                    borderValue=(255,255,255))
    return rotated_image

In [8]:
# Color conversion
def getGreyScaleImage( img ): 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
#     cv2.imshow("grey", img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return img

In [9]:
# Image Thresholding
def getThreshImage( img ):  
    ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )
#     cv2.imshow("thresh", thresh)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return thresh

In [10]:
# Morphological Transformation
def getErosionImage( img ):
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = 1)
#     cv2.imshow("closing", erosion)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return erosion

In [11]:
# Invert binarize the image
def invertImage( img ):
    img_blur = cv2.GaussianBlur(img, (3,3), 0)
    canny_edges = cv2.Canny(img_blur, 10, 70)
    ret, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    return mask

In [12]:
# checking the functions
img = cv2.imread('handA.png')
erosion = getErosionImage(img)
grey = getGreyScaleImage(img)
thresh = getThreshImage(grey)
cv2.imshow("img", invertImage(img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
#define detecting the edges
def edgeDetection( img ):
    # Convert to graycsale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
    # Sobel Edge Detection on the X axis
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=1) 
     # Sobel Edge Detection on the Y axis
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=1)
    # Combined X and Y Sobel Edge Detection
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=1) 
    # Display Sobel Edge Detection Images
#     cv2.imshow('Sobel X', sobelx)
#     cv2.waitKey(0)
#     cv2.imshow('Sobel Y', sobely)
#     cv2.waitKey(0)
#     cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
#     cv2.waitKey(0)
    
    # Canny Edge Detection
    edges = cv2.Canny(image=img_blur, threshold1=20, threshold2=70)
    # Display Canny Edge Detection Image
#     cv2.imshow('Canny Edge Detection', edges)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return invertImage(edges)

In [14]:
#checking the functions
img = cv2.imread('handA.png') 
img = edgeDetection(img)
img = invertImage(img)
cv2.imshow("img", (img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
# video cam detection
# cam = cv2.VideoCapture(0)

# while True :
#     ret, frame = cam.read()
#     cv2.imshow('Live', edgeDetection(frame))
#     if( cv2.waitKey(1) == 13 ): #enter key
#         break

# cam.release()
# cv2.destroyAllWindows()

In [27]:
def getContours( gray ):
#     gray = getGreyScaleImage(img)
    edged = cv2.Canny(gray, 30, 200) 
    
    #black image
    blk = np.zeros((img.shape[0], img.shape[1], 3))

    # finding contours 
    # contours alter image, hence use its copy
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [17]:
# img = cv2.imread('handP.png')
# gray = getGreyScaleImage(img)
# edged = cv2.Canny(gray, 30, 200)
# cv2.imshow('canny', edged)
# cv2.waitKey(0)

# #black image
# blk = np.zeros((img.shape[0], img.shape[1], 3))

# # finding contours 
# # contours alter image, hence use its copy
# contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# cv2.imshow('Canny edges after Contour', edged)
# cv2.waitKey(0)
# cv2.drawContours(blk, contours, -1, (0,255,0), 3)
# cv2.imshow('Contours', blk)
# cv2.waitKey(0)
# print(str(len(contours)))
# cv2.destroyAllWindows() 

In [38]:
# Approximating Contours

img = cv2.imread('handA.png')
gray = getGreyScaleImage(img)

# use thresh image if output is good
thresh = getThreshImage(getGreyScaleImage(img))

contours = getContours(getGreyScaleImage(img))

for c in contours:   
    x,y,h,w = cv2.boundingRect(c)
#     cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
#     cv2.imshow('bounding rect', img)
    
    
for c in contours:  
    accuracy = 0.01 * cv2.arcLength(c, True) 
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(img, [approx], 0, (255,0,0), 2)
    cv2.imshow('Approx', img)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
# Convex Hull

img = cv2.imread('handP.png')
gray = getGreyScaleImage(img)

# use thresh image if output is good
thresh = getThreshImage(getGreyScaleImage(img))

contours = getContours(getGreyScaleImage(img))

# removing the largest sized contour
n = len(contours) - 1
contours = sorted(contours, key=cv2.contourArea, reverse=False)[:n]

for c in contours: 
    hull = cv2.convexHull(c) 
    cv2.drawContours(img, [hull], 0, (0,255,0), 3)
#     cv2.imshow('Convex Hull', img)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [113]:
# detecting Squares

image = cv2.imread('handP.png')
gray = getGreyScaleImage(image)

contours = getContours(getGreyScaleImage(image))

for cnt in contours:
    
    # Get approximate polygons
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt,True),True)
    
    if len(approx) == 4 :
        x,y,w,h = cv2.boundingRect(cnt)
        M = cv2.moments(cnt) 
        
        # Check to see if 4-side polygon is square
        # cv2.boundingRect returns the top left and then width and height
        if abs(w-h) <= 20: 
            cv2.drawContours(image, [cnt], 0, (0, 125 ,255), -1)  
        
cv2.imshow('Shape',image)
cv2.waitKey(0) 
cv2.destroyAllWindows()